### Imports and Path setup

In [1]:
from pathlib import Path
import chromadb
import pickle
import os
from dotenv import load_dotenv
load_dotenv()

multiquery_rag_output_path = "../RAG Results/multiquery_rag_results.txt"
Relative_Database_path = "./chroma_Data_v4"
Absolute_Database_path = Path(Relative_Database_path).resolve()
file_path = "../Chunking/Chunk_files/julius-caesar_chunks_semantic.pkl"
# Create a new collection with a unique name
collection_name = "anlp_rag_collection"
# # Set API key
# os.environ["GOOGLE_API_KEY"] = os.environ.get("GEMINI_API_KEY")


### Chroma Setup and Chunk Loading
Sets up persistant client and loads previously computed chunks

In [2]:
# Initialize the persistent client
client = chromadb.PersistentClient(path=Absolute_Database_path)
print(f"[INFO] ChromaDB client initialized at: {Absolute_Database_path}")

# List existing collections
existing_collections = client.list_collections()
print(f"Existing collections: {[c.name for c in existing_collections]}")

PanicException: range start index 10 out of range for slice of length 9

In [ ]:
# Load chunks with version compatibility handling

loaded_docs = []

try:
    with open(file_path, "rb") as f:
        # Try loading normally first
        try:
            loaded_docs = pickle.load(f)
            print(f"Successfully loaded {len(loaded_docs)} chunks from '{file_path}'.")
        except AttributeError as e:
            # Handle Pydantic v1 vs v2 incompatibility
            if "__fields_set__" in str(e):
                print("[WARN] Pickle file has version incompatibility. Attempting to reconstruct...")
                
                # Reopen and load with custom unpickler
                f.seek(0)
                import pickle
                
                # Create a custom unpickler to handle missing attributes
                class CompatibilityUnpickler(pickle.Unpickler):
                    def find_class(self, module, name):
                        # Map old LangChain classes to current ones
                        if 'langchain' in module and 'schema' in module:
                            # Import current Document class
                            from langchain.schema import Document
                            if name == 'Document':
                                return Document
                        return super().find_class(module, name)
                
                loaded_docs = CompatibilityUnpickler(f).load()
                
                # Reconstruct documents if needed
                from langchain.schema import Document
                reconstructed_docs = []
                for doc in loaded_docs:
                    if hasattr(doc, 'page_content') and hasattr(doc, 'metadata'):
                        # Create fresh Document object
                        new_doc = Document(
                            page_content=doc.page_content,
                            metadata=doc.metadata if isinstance(doc.metadata, dict) else {}
                        )
                        reconstructed_docs.append(new_doc)
                    else:
                        reconstructed_docs.append(doc)
                
                loaded_docs = reconstructed_docs
                print(f"[SUCCESS] Reconstructed {len(loaded_docs)} chunks successfully.")
            else:
                raise
                
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"Error loading file: {e}")
    print(f"Error type: {type(e).__name__}")
    
    # Last resort: regenerate the chunks
    print("\n[TIP] If error persists, regenerate the chunks using First_chunking_attempt.ipynb")

# Verify loaded documents
print("\nVerification:")
if loaded_docs:
    print(f"✓ Total chunks loaded: {len(loaded_docs)}")
    print(f"✓ First chunk preview: {loaded_docs[0].page_content[:100]}...")
    print(f"✓ Metadata: {loaded_docs[0].metadata}")
else:
    print("✗ No documents loaded. Please regenerate chunks.")


Successfully loaded 126 chunks from '../Chunking/Chunk_files/julius-caesar_chunks_semantic.pkl'.

Verification:
✓ Total chunks loaded: 126
✓ First chunk preview: Michael Witmore
Director, Folger Shakespeare Library
It is hard to imagine a world without Shakespea...
✓ Metadata: {'source': '../julius-caesar_PDF_FolgerShakespeare.pdf', 'page_number': 3, 'c': 'semantic', 'ischunk': True}


### Set up Embedding Function
Will use default SentenceTransformer for generating embeddings

In [ ]:
# Install if needed
# !pip install sentence_transformers

# Set up embedding function
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
print("Embedding function initialized with model: all-MiniLM-L6-v2")

Embedding function initialized with model: all-MiniLM-L6-v2


### Creating new Collection

In [ ]:
from datetime import datetime

# FORCE DELETE the collection if it exists
try:
    client.delete_collection(name=collection_name)
    print(f"[INFO] Deleted existing collection '{collection_name}'")
except Exception as e:
    print(f"[INFO] No existing collection named '{collection_name}' to delete.")

# Create a FRESH collection
collection = client.create_collection(
    name=collection_name,
    embedding_function=embedding_function,
    metadata={
        "description": "Julius Caesar Chunks collection for RAG",
        "created": str(datetime.now())
    }
)

print(f"[SUCCESS] Fresh collection '{collection_name}' created successfully")
print(f"Current count in collection: {collection.count()}")

[INFO] Deleted existing collection 'anlp_rag_collection'
[SUCCESS] Fresh collection 'anlp_rag_collection' created successfully
Current count in collection: 0


### Add data to collection
The chunks have to be given an id and added to the collection now

In [ ]:
import uuid
import hashlib

# Extract document name from file path for ID generation
# This will give us "julius-caesar" from the file path
doc_name = file_path.split('/')[-1].split('_chunks')[0]

# Prepare documents for ChromaDB
ids = []
documents = []
metadatas = []

print(f"[INFO] Generating IDs with prefix: {doc_name}")

# Process each loaded document chunk
for i, doc in enumerate(loaded_docs):
    # Generate a deterministic ID based on document name and index
    # This ensures all chunks from the same document have consistent IDs
    doc_id = f"{doc_name}_chunk_{i}"
    
    # Get the document text
    document_text = doc.page_content
    
    # Get the document metadata
    metadata = doc.metadata
    
    # Add to our lists
    ids.append(doc_id)
    documents.append(document_text)
    metadatas.append(metadata)

print(f"[INFO] Prepared {len(ids)} documents with IDs like: {ids[0]}, {ids[1] if len(ids) > 1 else 'N/A'}...")

# Add documents in batches to avoid memory issues
batch_size = 500
total_added = 0

for i in range(0, len(ids), batch_size):
    end_idx = min(i + batch_size, len(ids))
    
    # Simply add all documents (collection is fresh, no need to update)
    collection.add(
        ids=ids[i:end_idx],
        documents=documents[i:end_idx],
        metadatas=metadatas[i:end_idx]
    )
    
    total_added += end_idx - i
    print(f"[INFO] Added batch: {i} to {end_idx-1} ({end_idx-i} documents)")

print(f"\n[SUCCESS] Added {total_added} documents to collection '{collection_name}'")
print(f"[INFO] All chunks have IDs in format: {doc_name}_chunk_<number>")

[INFO] Generating IDs with prefix: julius-caesar
[INFO] Prepared 126 documents with IDs like: julius-caesar_chunk_0, julius-caesar_chunk_1...
[INFO] Added batch: 0 to 125 (126 documents)

[SUCCESS] Added 126 documents to collection 'anlp_rag_collection'
[INFO] All chunks have IDs in format: julius-caesar_chunk_<number>
[INFO] Added batch: 0 to 125 (126 documents)

[SUCCESS] Added 126 documents to collection 'anlp_rag_collection'
[INFO] All chunks have IDs in format: julius-caesar_chunk_<number>


In [ ]:
# Check collection count
count = collection.count()
print(f"Total documents in collection: {count}")

# Peek at the first few entries
peek = collection.peek(limit=3)
print("\nSample entries:")
for i, (doc_id, doc_text, metadata) in enumerate(zip(
    peek['ids'], peek['documents'], peek['metadatas']
)):
    print(f"\n--- Document {i+1} ---")
    print(f"ID: {doc_id}")
    print(f"Text: {doc_text[:100]}...")
    print(f"Metadata: {metadata}")

Total documents in collection: 126

Sample entries:

--- Document 1 ---
ID: julius-caesar_chunk_0
Text: Michael Witmore
Director, Folger Shakespeare Library
It is hard to imagine a world without Shakespea...
Metadata: {'source': '../julius-caesar_PDF_FolgerShakespeare.pdf', 'page_number': 3, 'ischunk': True, 'c': 'semantic'}

--- Document 2 ---
ID: julius-caesar_chunk_1
Text: Until now, with the release of The Folger Shakespeare (formerly
Folger Digital Texts), readers in se...
Metadata: {'source': '../julius-caesar_PDF_FolgerShakespeare.pdf', 'c': 'semantic', 'page_number': 4, 'ischunk': True}

--- Document 3 ---
ID: julius-caesar_chunk_2
Text: At
any point in the text, you can hover your cursor over a bracket for
more information. Because the...
Metadata: {'c': 'semantic', 'page_number': 5, 'source': '../julius-caesar_PDF_FolgerShakespeare.pdf', 'ischunk': True}


### Querying the Database

In [ ]:
# Rich table for displaying results (optional but nice)
try:
    from rich.console import Console
    from rich.table import Table
    
    console = Console()
    use_rich = True
except ImportError:
    use_rich = False
    print("Rich package not found. Using standard print.")

# Function to display query results
def print_results(results, use_rich=use_rich):
    if use_rich:
        table = Table(show_header=True, header_style="bold magenta")
        table.add_column("Rank", width=6)
        table.add_column("Document ID")
        table.add_column("Document Text", width=60)
        table.add_column("Page")
        table.add_column("Distance")
        
        docs = results['documents'][0]
        ids = results['ids'][0]
        metas = results['metadatas'][0]
        distances = results['distances'][0]
        
        for i, (doc, doc_id, meta, dist) in enumerate(zip(docs, ids, metas, distances)):
            table.add_row(
                str(i+1),
                doc_id,
                (doc[:100] + "...") if len(doc) > 100 else doc,
                str(meta.get('page_number', 'N/A')),
                f"{dist:.4f}"
            )
        
        console.print(table)
    else:
        # Standard print version
        for i, (doc, meta, dist) in enumerate(zip(
            results['documents'][0], 
            results['metadatas'][0], 
            results['distances'][0]
        )):
            print(f"\n--- Result {i+1} ---")
            print(f"Text: {doc[:100]}...")
            print(f"Metadata: {meta}")
            print(f"Distance: {dist:.4f}")

In [ ]:
# Run a sample query
query = "What themes are explored in Julius Caesar?"
results = collection.query(
    query_texts=[query],
    n_results=3,
    include=["documents", "metadatas", "distances"]
)

print(f"\nResults for query: '{query}'")
print_results(results)


Results for query: 'What themes are explored in Julius Caesar?'


┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━┓
┃ Rank   ┃ Document ID           ┃ Document Text                                                ┃ Page ┃ Distance ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━┩
│ 1      │ julius-caesar_chunk_… │ 95                                                           │ 51   │ 0.3517   │
│        │                       │ Julius Caesar                                                │      │          │
│        │                       │ ACT 3. SC. 1                                                 │      │          │
│        │                       │ POPILIUS                                                     │      │          │
│        │                       │ CASSIUS                                                      │      │          │
│        │                       │ POPILIUS                                                     │      │          │
│        │                       │ He walks away. BRUTUS                                        │      │          │
│        │                       │ CASSIUS                                                      │      │          │
│        │                       │ BRUTUS                                                       │      │          │
│        │                       │ CASSIUS...                                                   │      │          │
│ 2      │ julius-caesar_chunk_… │ CAESAR                                                       │ 50   │ 0.3766   │
│        │                       │ SOOTHSAYER                                                   │      │          │
│        │                       │ ARTEMIDORUS                                                  │      │          │
│        │                       │ DECIUS                                                       │      │          │
│        │                       │ ARTEMIDORUS                                                  │      │          │
│        │                       │ CAESAR                                                       │      │          │
│        │                       │ ARTEMIDORUS                                                  │      │          │
│        │                       │ CAESAR                                                       │      │          │
│        │                       │ PUBLIUS                                                      │      │          │
│        │                       │ CASSIUS                                                      │      │          │
│        │                       │ Caesar go...                                                 │      │          │
│ 3      │ julius-caesar_chunk_… │ 85                                                           │ 47   │ 0.3788   │
│        │                       │ Julius Caesar                                                │      │          │
│        │                       │ ACT 2. SC. 4                                                 │      │          │
│        │                       │ ARTEMIDORUS                                                  │      │          │
│        │                       │  Thy lover,                                                  │      │          │
│        │                       │  Artemidorus                                                 │      │          │
│        │                       │ He exits. PORTIA                                             │      │          │
│        │                       │ LUCIUS                                                       │      │          │
│        │                       │ PORTIA                                                       │      │          │
│        │                       │ En...                                                        │      │          │
└────────┴───────────────────────┴──────────────────────

### Natural Language Generation

In [ ]:
# !pip install groq --quiet

In [ ]:
# Fix version conflicts - downgrade to compatible versions
# !pip uninstall langchain-groq -y
# !pip install "langchain-core<0.3.0" --quiet

In [ ]:
import os
from groq import Groq

# Set the API key for Groq
os.environ["GROQ_API_KEY"] = "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"

# Initialize Groq client
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])

# Create a simple LLM wrapper that works like LangChain's ChatGroq
class GroqLLM:
    def __init__(self, model, temperature=0.7):
        self.model = model
        self.temperature = temperature
        self.client = groq_client
    
    def invoke(self, prompt):
        """Invoke the LLM with a prompt and return response"""
        # Handle both string prompts and LangChain PromptValue objects
        if hasattr(prompt, 'to_string'):
            prompt_text = prompt.to_string()
        elif hasattr(prompt, 'text'):
            prompt_text = prompt.text
        else:
            prompt_text = str(prompt)
        
        response = self.client.chat.completions.create(
            messages=[{"role": "user", "content": prompt_text}],
            model=self.model,
            temperature=self.temperature
        )
        
        # Return an object with .content attribute for compatibility
        class Response:
            def __init__(self, content):
                self.content = content
            def __str__(self):
                return self.content
        
        return Response(response.choices[0].message.content)

# Initialize our custom LLM wrapper
llm = GroqLLM(
    model="llama-3.3-70b-versatile",
    temperature=0.7
)

print("Groq LLM initialized with model: llama-3.3-70b-versatile (using native Groq SDK)")

Groq LLM initialized with model: llama-3.3-70b-versatile (using native Groq SDK)


In [ ]:
from langchain_core.prompts import PromptTemplate

# Better prompt template for Julius Caesar
rag_prompt_template = """
You are an expert on Shakespeare's Julius Caesar. Answer questions using ONLY the context below.
If you can't find a complete answer in the context but see partial information, try to provide what you can find and acknowledge the limitations of the available information.
If there is NO relevant information at all in the context, respond with "I don't have enough information to answer this question."

Context:
{context}

Question: {query}

Answer (based only on the context provided):
"""

prompt = PromptTemplate(
    template=rag_prompt_template,
    input_variables=["context", "query"]
)

In [ ]:
# !pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi
import numpy as np

def answer_with_hybrid_rag(query, n_results=5):
    # 1. Semantic search with ChromaDB
    semantic_results = collection.query(
        query_texts=[query],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    
    # 2. Perform keyword search with BM25
    # First get all documents to search across
    all_docs = collection.get(
        limit=100,  # Adjust based on your collection size
        include=["documents", "metadatas"]
    )
    
    # Tokenize for BM25
    tokenized_docs = [doc.split() for doc in all_docs["documents"]]
    bm25 = BM25Okapi(tokenized_docs)
    
    # Get BM25 scores
    tokenized_query = query.split()
    bm25_scores = bm25.get_scores(tokenized_query)
    
    # Get top BM25 results
    top_bm25_indices = np.argsort(bm25_scores)[-n_results:][::-1]
    
    # 3. Combine results (simple union)
    combined_docs = []
    combined_meta = []
    combined_ids = [] 
    seen_ids = set()
    
    # Add semantic results
    for doc, meta, doc_id in zip(
        semantic_results["documents"][0], 
        semantic_results["metadatas"][0],
        semantic_results["ids"][0]
    ):
        if doc_id not in seen_ids:
            combined_docs.append(doc)
            combined_meta.append(meta)
            combined_ids.append(doc_id)
            seen_ids.add(doc_id)
    
    # Add keyword results
    for idx in top_bm25_indices:
        doc_id = all_docs["ids"][idx]
        if doc_id not in seen_ids:
            combined_docs.append(all_docs["documents"][idx])
            combined_meta.append(all_docs["metadatas"][idx])
            combined_ids.append(doc_id)
            seen_ids.add(doc_id)
    
    # Limit to n_results total
    combined_docs = combined_docs[:n_results]
    combined_meta = combined_meta[:n_results]
    combined_ids = combined_ids[:n_results]
    
    # Format context and complete RAG as before
    formatted_docs = []
    for doc, meta in zip(combined_docs, combined_meta):
        page_num = meta.get("page_number", "unknown")
        formatted_docs.append(f"[Page {page_num}]: {doc}")
    
    context = "\n\n---\n\n".join(formatted_docs)
    filled_prompt = prompt.format(context=context, query=query)
    response = llm.invoke(filled_prompt)
    
    # Create a mock results object for print_results compatibility
    mock_results = {
        "documents": [combined_docs],
        "metadatas": [combined_meta],
        "distances": [[0.0] * len(combined_docs)],  # Placeholder distances
        "ids": [combined_ids]
    }
    
    return {
        "query": query,
        "answer": response.content if hasattr(response, 'content') else str(response),
        "source_documents": mock_results
    }

In [ ]:
# Test our RAG pipeline with a question
test_query = "What is the relationship between Brutus and Caesar?"
response = answer_with_hybrid_rag(test_query)

print(f"Question: {test_query}")
print(f"\nAnswer: {response['answer']}")
print("\nSources:")
print_results(response["source_documents"])

Question: What is the relationship between Brutus and Caesar?

Answer: According to the context, Brutus and Caesar have a close relationship. In Act 3, Scene 1, Antony says that Caesar loved Brutus dearly, referring to him as Caesar's "angel" (Page 68). Additionally, Brutus is described as having stabbed Caesar, which suggests a level of intimacy and betrayal (Page 68). Furthermore, in Act 2, Scene 4, a warning is given to Caesar to "beware of Brutus" (Page 47), implying that Brutus poses a threat to Caesar. Overall, the relationship between Brutus and Caesar appears to be complex, with Brutus being both close to and a threat to Caesar.

Sources:


┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━┓
┃ Rank   ┃ Document ID           ┃ Document Text                                                ┃ Page ┃ Distance ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━┩
│ 1      │ julius-caesar_chunk_… │ 23                                                           │ 17   │ 0.0000   │
│        │                       │ Julius Caesar                                                │      │          │
│        │                       │ ACT 1. SC. 2                                                 │      │          │
│        │                       │ BRUTUS                                                       │      │          │
│        │                       │  “Brutus” and “Caesar”—what should be in that                │      │          │
│        │                       │  “Caesar”? Why sh...                                         │      │          │
│ 2      │ julius-caesar_chunk_… │ 85                                                           │ 47   │ 0.0000   │
│        │                       │ Julius Caesar                                                │      │          │
│        │                       │ ACT 2. SC. 4                                                 │      │          │
│        │                       │ ARTEMIDORUS                                                  │      │          │
│        │                       │  Thy lover,                                                  │      │          │
│        │                       │  Artemidorus                                                 │      │          │
│        │                       │ He exits. PORTIA                                             │      │          │
│        │                       │ LUCIUS                                                       │      │          │
│        │                       │ PORTIA                                                       │      │          │
│        │                       │ En...                                                        │      │          │
│ 3      │ julius-caesar_chunk_… │ Bear back! If you have tears, prepare to shed them now. You  │ 68   │ 0.0000   │
│        │                       │ all do know this mantle. I remember                          │      │          │
│        │                       │  The...                                                      │      │          │
│ 4      │ julius-caesar_chunk_… │ 109                                                          │ 58   │ 0.0000   │
│        │                       │ Julius Caesar                                                │      │          │
│        │                       │ ACT 3. SC. 1                                                 │      │          │
│        │                       │ CASSIUS                                                      │      │          │
│        │                       │ ANTONY                                                       │      │          │
│        │                       │ CASSIUS                                                      │      │          │
│        │                       │ ANTONY                                                       │      │          │
│        │                       │ BRUTUS                                                       │      │          │
│        │                       │  Either a coward or a flatterer....                          │      │          │
│ 5      │ julius-caesar_chunk_… │ 199                                                          │ 101  │ 0.0000   │
│        │                       │ Julius Caesar                                                │      │          │
│        │                       │ ACT 5. SC. 4                                                 │      │          │
│        │                       │ They exit. BRUTUS    

In [ ]:
# Test with multiple questions to evaluate system
results_for_export = []

test_questions = [
    "What are the main themes in Julius Caesar?",
    "How does Brutus justify killing Caesar?",
    "What role does Cassius play in the conspiracy?"
]

for question in test_questions:
    print("\n" + "="*50)
    print(f"Question: {question}")
    response = answer_with_hybrid_rag(question)
    print(f"\nAnswer: {response['answer']}")
    print("\nTop source:")
    if len(response["source_documents"]["documents"][0]) > 0:
        top_doc = response["source_documents"]["documents"][0][0]
        top_meta = response["source_documents"]["metadatas"][0][0]
        page = top_meta.get("page_number", "N/A")
        print(f"[Page {page}]:\n{top_doc[:200]}...")  # Print first 200 chars
        # Save for export
        results_for_export.append({
            "question": question,
            "answer": response['answer'],
            "page": page,
            "chunk": top_doc
        })
    else:
        print("No sources found.")
        results_for_export.append({
            "question": question,
            "answer": response['answer'],
            "page": None,
            "chunk": None
        })

# Export results to a well-formatted text file
with open(multiquery_rag_output_path, "w", encoding="utf-8") as f:
    f.write("RAG Multi-Query Evaluation Results\n")
    f.write("="*60 + "\n\n")
    for idx, res in enumerate(results_for_export, 1):
        f.write(f"Question {idx}: {res['question']}\n")
        f.write(f"Answer:\n{res['answer']}\n\n")
        if res["chunk"]:
            f.write(f"Top Source Chunk (Page {res['page']}):\n{res['chunk']}\n")
        else:
            f.write("Top Source Chunk: No sources found.\n")
        f.write("-"*60 + "\n\n")
print(f"\nResults exported to {multiquery_rag_output_path}")


Question: What are the main themes in Julius Caesar?

Answer: I don't have enough information to answer this question. The context provided only includes excerpts from specific scenes in Julius Caesar and does not provide a comprehensive overview of the play's themes.

Top source:
[Page 51]:
95
Julius Caesar
ACT 3. SC. 1
POPILIUS
CASSIUS
POPILIUS
He walks away. BRUTUS
CASSIUS
BRUTUS
CASSIUS
BRUTUS
CASSIUS
Trebonius and Antony exit. DECIUS
BRUTUS
CINNA
CAESAR
METELLUS
, to Cassius
 
 I wis...

Question: How does Brutus justify killing Caesar?

Answer: I don't have enough information to answer this question. The context provided only includes excerpts from specific scenes in Julius Caesar and does not provide a comprehensive overview of the play's themes.

Top source:
[Page 51]:
95
Julius Caesar
ACT 3. SC. 1
POPILIUS
CASSIUS
POPILIUS
He walks away. BRUTUS
CASSIUS
BRUTUS
CASSIUS
BRUTUS
CASSIUS
Trebonius and Antony exit. DECIUS
BRUTUS
CINNA
CAESAR
METELLUS
, to Cassius
 
 I wis...

Questi

In [ ]:
# !pip install "ragas>=0.2.0,<0.3.0" --quiet

ERROR: Cannot install langchain-community==0.2.17, langchain-core==0.2.38 and langchain==0.2.16 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
# === Groq + RAG + RAGAS Evaluation ===
# Prereqs:
# pip install ragas datasets groq tqdm sentence-transformers numpy

import os
import json
import numpy as np
import time
import asyncio
from datetime import datetime
from tqdm import tqdm
from datasets import Dataset
from groq import Groq

from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from ragas.embeddings.base import HuggingfaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_core.prompt_values import PromptValue
from langchain_core.outputs import Generation, LLMResult


# ==== CONFIG ====
# Use the API key already set in previous cell
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])

testbed_path = "../RAG Results/test_bed.json"
output_metrics_path = "../RAG Results/multiquery_rag_metrics.txt"
cached_answers_path = "../RAG Results/cached_rag_answers.json"  # NEW: Cache file
TOP_K = 3

GROQ_RAG_MODEL = "llama-3.3-70b-versatile"
GROQ_RAGAS_MODEL = "llama-3.3-70b-versatile"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# Rate limiting config for llama-3.3-70b-versatile:
# RPM: 30 (requests per minute)
# RPD: 1,000 (requests per day)
# TPM: 12,000 (tokens per minute)
# TPD: 100,000 (tokens per day)
REQUEST_DELAY = 2.5  # seconds between requests (allows ~24 RPM, safe margin below 30 RPM)
BATCH_SIZE = 5  # Process in small batches to avoid hitting token limits
MAX_RETRIES = 3  # Retry failed requests

print("Exists:", os.path.exists(testbed_path))
print("Size:", os.path.getsize(testbed_path), "bytes")

with open(testbed_path, "r", encoding="utf-8") as f:
    first_200 = f.read(200)
print("First few characters:\n", first_200)


# ==== 1️⃣ Load test data ====
with open(testbed_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

print(f"[INFO] Loaded {len(test_data)} QA pairs from testbed.")


# ==== 2️⃣ Groq generation with retry logic ====
def generate_with_groq(prompt, model_name=GROQ_RAG_MODEL, retries=MAX_RETRIES):
    for attempt in range(retries):
        try:
            chat_completion = groq_client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model=model_name,
                temperature=0.7,
            )
            time.sleep(REQUEST_DELAY)
            return chat_completion.choices[0].message.content.strip()
        except Exception as e:
            if "rate_limit" in str(e).lower():
                wait_time = REQUEST_DELAY * (attempt + 2)  # Exponential backoff
                print(f"[WARN] Rate limit hit. Waiting {wait_time}s before retry {attempt + 1}/{retries}")
                time.sleep(wait_time)
            else:
                print(f"[ERROR] Groq API call failed (attempt {attempt + 1}): {e}")
                if attempt == retries - 1:
                    time.sleep(REQUEST_DELAY)
                    return None
                time.sleep(REQUEST_DELAY)
    return None


# ==== 3️⃣ Groq wrapper for RAGAS following BaseRagasLLM interface ====
from ragas.llms.base import BaseRagasLLM as RagasBaseLLM
from ragas.run_config import RunConfig

class GroqRagasLLM(RagasBaseLLM):
    """Groq LLM wrapper implementing RAGAS BaseRagasLLM interface."""
    
    def __init__(self, model_name):
        super().__init__(run_config=RunConfig())
        self.model_name = model_name
        self.client = Groq(api_key=os.environ["GROQ_API_KEY"])
        self.request_count = 0
        self.last_request_time = time.time()

    def _rate_limit_check(self):
        """Ensure we don't exceed rate limits"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        
        # Ensure minimum delay between requests
        if time_since_last < REQUEST_DELAY:
            sleep_time = REQUEST_DELAY - time_since_last
            time.sleep(sleep_time)
        
        self.last_request_time = time.time()
        self.request_count += 1

    def _extract_text_from_prompt(self, prompt: PromptValue) -> str:
        """Extract text from PromptValue object."""
        # PromptValue has .to_string() method
        if hasattr(prompt, "to_string"):
            return prompt.to_string()
        # Fallback to string conversion
        return str(prompt)

    def generate_text(
        self,
        prompt: PromptValue,
        n: int = 1,
        temperature: float = 0.01,
        stop=None,
        callbacks=None,
    ) -> LLMResult:
        """Synchronous generation - required by BaseRagasLLM."""
        prompt_text = self._extract_text_from_prompt(prompt)
        generations = []
        
        for i in range(n):
            for attempt in range(MAX_RETRIES):
                try:
                    # Rate limit check
                    self._rate_limit_check()
                    
                    chat_completion = self.client.chat.completions.create(
                        messages=[{"role": "user", "content": prompt_text}],
                        model=self.model_name,
                        temperature=temperature,
                    )
                    
                    text = chat_completion.choices[0].message.content.strip()
                    generations.append([Generation(text=text)])
                    break  # Success
                    
                except Exception as e:
                    if "rate_limit" in str(e).lower() and attempt < MAX_RETRIES - 1:
                        wait_time = REQUEST_DELAY * (attempt + 2)
                        print(f"[WARN] Rate limit hit. Waiting {wait_time}s (attempt {attempt + 1})")
                        time.sleep(wait_time)
                    else:
                        print(f"[ERROR] Failed (attempt {attempt + 1}): {e}")
                        if attempt == MAX_RETRIES - 1:
                            generations.append([Generation(text=f"[Error: {e}]")])
                        else:
                            time.sleep(REQUEST_DELAY)
        
        return LLMResult(generations=generations)

    async def agenerate_text(
        self,
        prompt: PromptValue,
        n: int = 1,
        temperature: float = 0.01,
        stop=None,
        callbacks=None,
    ) -> LLMResult:
        """Asynchronous generation - required by BaseRagasLLM."""
        prompt_text = self._extract_text_from_prompt(prompt)
        generations = []
        
        for i in range(n):
            for attempt in range(MAX_RETRIES):
                try:
                    # Rate limit check
                    await asyncio.sleep(REQUEST_DELAY)
                    
                    # Run blocking SDK call in thread
                    chat_completion = await asyncio.to_thread(
                        self.client.chat.completions.create,
                        messages=[{"role": "user", "content": prompt_text}],
                        model=self.model_name,
                        temperature=temperature,
                    )
                    
                    text = chat_completion.choices[0].message.content.strip()
                    generations.append([Generation(text=text)])
                    break  # Success
                    
                except Exception as e:
                    if "rate_limit" in str(e).lower() and attempt < MAX_RETRIES - 1:
                        wait_time = REQUEST_DELAY * (attempt + 2)
                        print(f"[WARN] Rate limit hit. Waiting {wait_time}s (attempt {attempt + 1})")
                        await asyncio.sleep(wait_time)
                    else:
                        print(f"[ERROR] Failed (attempt {attempt + 1}): {e}")
                        if attempt == MAX_RETRIES - 1:
                            generations.append([Generation(text=f"[Error: {e}]")])
                        else:
                            await asyncio.sleep(REQUEST_DELAY)
        
        return LLMResult(generations=generations)

    def is_finished(self, response: LLMResult) -> bool:
        """Check if response is complete - required by BaseRagasLLM."""
        return True


# ==== 4️⃣ Check collection availability ====
try:
    collection.query(query_texts=["test"], n_results=1)
except NameError:
    print("\n[CRITICAL WARNING] The 'collection' object (ChromaDB) is NOT defined.")
    print("Please initialize your ChromaDB client/collection before running this cell.")
    raise SystemExit


# ==== 5️⃣ Generate records with caching and rate limiting ====
records = []

# Check if cached answers exist
if os.path.exists(cached_answers_path):
    print(f"[INFO] Found cached answers at '{cached_answers_path}'")
    try:
        with open(cached_answers_path, "r", encoding="utf-8") as f:
            cached_data = json.load(f)
        
        # Validate cache matches current test data
        if len(cached_data) == len(test_data):
            questions_match = all(
                cached_data[i]["question"] == test_data[i]["question"] 
                for i in range(len(test_data))
            )
            
            if questions_match:
                print(f"[INFO] Loading {len(cached_data)} cached answers (skipping generation)")
                records = cached_data
            else:
                print("[WARN] Cached questions don't match test data. Regenerating...")
        else:
            print(f"[WARN] Cache size mismatch ({len(cached_data)} vs {len(test_data)}). Regenerating...")
    except Exception as e:
        print(f"[ERROR] Failed to load cache: {e}. Regenerating...")

# Generate new answers if cache not usable
if not records:
    print(f"[INFO] Generating RAG answers with rate limiting (max 30 RPM)...")
    print(f"[INFO] Request delay: {REQUEST_DELAY}s | Batch size: {BATCH_SIZE}")
    
    for item in tqdm(test_data, desc="Generating Groq RAG answers"):
        question = item["question"]
        ideal_answer = item["ideal_answer"]

        retrieved = collection.query(query_texts=[question], n_results=TOP_K)
        retrieved_docs = retrieved["documents"][0]
        retrieved_context = "\n".join(retrieved_docs)

        prompt = (
            f"Context:\n{retrieved_context}\n\n"
            f"Question:\n{question}\n\nAnswer:"
        )

        generated_answer = generate_with_groq(prompt)
        if not generated_answer:
            generated_answer = f"[Fallback mock answer] Context excerpt: {retrieved_docs[0][:150]}..."

        records.append({
            "question": question,
            "contexts": retrieved_docs,
            "answer": generated_answer,
            "ground_truth": ideal_answer,
        })
        
        # Progress update every 5 questions
        if len(records) % 5 == 0:
            print(f"[INFO] Processed {len(records)}/{len(test_data)} questions")
    
    # Save generated answers to cache
    try:
        os.makedirs(os.path.dirname(cached_answers_path), exist_ok=True)
        with open(cached_answers_path, "w", encoding="utf-8") as f:
            json.dump(records, f, indent=2, ensure_ascii=False)
        print(f"[SUCCESS] Cached {len(records)} answers to '{cached_answers_path}'")
    except Exception as e:
        print(f"[WARN] Failed to save cache: {e}")


# ==== 6️⃣ Convert to HF Dataset ====
dataset = Dataset.from_list(records)
print(f"[INFO] Created dataset with {len(dataset)} samples")


# ==== 7️⃣ Custom HuggingFace Embedding Wrapper ====
class CustomHuggingfaceEmbeddings(HuggingfaceEmbeddings):
    """Implements both sync + async embedding methods for latest RAGAS."""
    def __init__(self, model_name: str):
        # ✅ Do not call super()
        self.model_name = model_name
        self.model = SentenceTransformer(model_name)

    # --- Sync methods ---
    def embed_documents(self, texts):
        return self.model.encode(texts, show_progress_bar=False).tolist()

    def embed_query(self, text):
        return self.model.encode([text], show_progress_bar=False).tolist()[0]

    # --- Async methods ---
    async def aembed_documents(self, texts):
        return self.embed_documents(texts)

    async def aembed_query(self, text):
        return self.embed_query(text)


# ==== 8️⃣ Evaluate with RAGAS ====
llm = GroqRagasLLM(GROQ_RAGAS_MODEL)
embeddings = CustomHuggingfaceEmbeddings(model_name=EMBED_MODEL)

print(f"\n[INFO] Starting RAGAS evaluation with {GROQ_RAGAS_MODEL}...")
print(f"[INFO] Rate limits: 30 RPM | 12K TPM | Using {REQUEST_DELAY}s delays")
print(f"[INFO] Estimated time: ~{len(dataset) * REQUEST_DELAY / 60:.1f} minutes")

start_time = time.time()

results = evaluate(
    dataset=dataset,
    metrics=[faithfulness, answer_relevancy],
    llm=llm,
    embeddings=embeddings
)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\n[SUCCESS] Evaluation completed in {elapsed_time / 60:.2f} minutes")


# ==== 9️⃣ Save Results ====
faithfulness_scores = results["faithfulness"]
answer_relevancy_scores = results["answer_relevancy"]

# ✅ Compute mean values
faithfulness_mean = float(np.mean(faithfulness_scores))
answer_relevancy_mean = float(np.mean(answer_relevancy_scores))

os.makedirs(os.path.dirname(output_metrics_path), exist_ok=True)

with open(output_metrics_path, "w", encoding="utf-8") as f:
    f.write("=== RAG Evaluation Metrics (Groq + RAGAS) ===\n")
    f.write(f"Timestamp: {datetime.now()}\n")
    f.write(f"Evaluation Duration: {elapsed_time / 60:.2f} minutes\n\n")
    f.write(f"RAG Generation Model: {GROQ_RAG_MODEL}\n")
    f.write(f"RAGAS Evaluation Model: {GROQ_RAGAS_MODEL}\n")
    f.write(f"Rate Limiting: {REQUEST_DELAY}s delay between requests\n")
    f.write(f"Cached Answers: {os.path.basename(cached_answers_path)}\n\n")
    f.write(f"Faithfulness (avg): {faithfulness_mean:.4f}\n")
    f.write(f"Answer Relevancy (avg): {answer_relevancy_mean:.4f}\n\n")
    f.write("Full Results:\n")
    f.write(str(results))

print(f"\n✅ Evaluation complete! Metrics saved to '{output_metrics_path}'")
print(f"Faithfulness (avg): {faithfulness_mean:.4f} | Answer Relevancy (avg): {answer_relevancy_mean:.4f}")
print(f"\n[TIP] To regenerate answers, delete: {cached_answers_path}")

ImportError: cannot import name 'is_data_content_block' from 'langchain_core.messages' (c:\Users\micro\Anaconda3\envs\NLP_2\Lib\site-packages\langchain_core\messages\__init__.py)